In [ ]:
#Instalação das bibliotecas necessárias
!pip install yfinance #Biblioteca para coleta de dados financeiros
!pip install crewai
!pip install 'crewai[tools]'
!pip install langchain #Biblioteca para processamento de linguagem natural
!pip install langchain-openai
!pip install python-dotenv #Biblioteca para manipulação de variáveis de ambiente

In [26]:
# Importação das bibliotecas necessárias
import json
import os
from datetime import datetime

import yfinance as yf
from crewai import Agent, Task
from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

In [19]:
# Função para coletar os dados do Yahoo Finance
def fectch_stock_history_price(ticket):
    stock = yf.download(ticket, start="2020-01-01", end="2020-12-31")
    return stock

# Criação da ferramenta para coletar os dados do Yahoo Finance
yahoo_finance_tool = Tool(
    name = "Yahoo Finance Tool",
    description = "This tool fetches stocks prices for {ticket} from some year about a specific stock from Yahoo Finance API.",
    func= lambda ticket: fectch_stock_history_price(ticket)
)

In [21]:
# Criando modelo de LLM usando OpenAI
load_dotenv() # Carrega as variáveis de ambiente
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [22]:
# Criando Agente que irá analisar os preços das ações
StockPriceAnalyst = Agent(
  role  = "Senior Stock Price Analyst",
  goal = "Find the {ticket} stock prices and analyses future prices and trends.",
  backstory = "You're a highly experienced stock price analyst with a deep understanding of the stock market and good prediction skills about this market.",
  verbose = True,
  llm = llm,
  max_iter = 5,
  memory = True,
  Tools = [yahoo_finance_tool]
)

In [28]:
getStockPrice = Task(
  description = "Analyse the stock {ticket} price history, create a trend analyses of up, down or sideways and predict future prices and trends.",
  expected_output = """Specify the current trend of the stock price and predict the future price and trend.
  eg. stock= 'APPL, price UP'
  """,
  agent = StockPriceAnalyst,
)